In [1]:
import cv2 
import numpy as np
import os # for filepaths
from matplotlib import pyplot as plt
import time
import mediapipe as mp # Para utilizar el modelo holistic

2024-10-22 00:06:37.345579: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/saur/anaconda3/envs/Lumiere/lib/python3.8/site-packages/cv2/../../lib64:
2024-10-22 00:06:37.345607: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mp_holistic = mp.solutions.holistic # Holistic Model, es un modelo que detecta landmarks faciales y corporales.
mp_drawing = mp.solutions.drawing_utils # Utils para dibujar los landmarks.

def mediapipe_detection(image, model): # Función que recibe una imagen (frame)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion 
    image.flags.writeable = False # image status
    results = model.process(image) #make pred
    image.flags.writeable = True # img status writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conv  
    return image, results 

In [3]:
# Función para dibujar landmarks, recibe una imagen(frame) y los resultados del modelo (holistic)
def draw_landmarks(image, results):
# ROSTRO
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image, # Se pasa el frame
            results.face_landmarks, # Se verifica si hay resultados correspondientes a la cara.
            mp_holistic.FACEMESH_TESSELATION, # Se pasan los nodos.
            mp_drawing.DrawingSpec(color=(209, 12, 163), thickness=1, circle_radius=1), # Nodos
            mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=1, circle_radius=1) # Líneas
        )
#POSE (BRAZOS, PECHO) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )
# MANO IZQUIERDA
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )
# MANO DERECHA
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(209,12,163), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)
        )

In [30]:
# cap = cv2.VideoCapture(0) #Device value 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
#         #Read Feed 
#         ret, frame = cap.read(); # return value and the frame(image)
    
#         #Make detections
#         image, results = mediapipe_detection(frame, holistic) # we use the frame for the function 
        
#         #draw landmarks
#         draw_landmarks(image, results) # Dibuja los landmarks en la imagen, a través de results.
        
#         #shown to screen
#         cv2.imshow("OpenCV Feed", image)
        
#         #for errors.
#         if cv2.waitKey(10) & 0xFF == ord("q"):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot o

In [41]:
def extract_keypoints(results): 
    # Por cada modelo (cuerpo, cara, mano izquierda y mano derecha, guarda un array por cada nodo, que contiene la información de sus coordenadas
    #x, y, z. Los nps zeros representan num_nodos_modelo * num_coordenadas. A pose, además, se le guarda la visibilidad.
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [45]:
extract_keypoints(results).shape # Nos devuelve un array unidimensional que contiene toda la información de los keypoints

(1662,)

In [59]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('NUEVO_MP_DATA') 

# Actions that we try to detect
actions = np.array(['hola', 'chao', 'mi_nombre_es', 'como_estas', 'buenas_noches', 'INSOR', 'bien', 'mal', 'por_favor', 'como_te_llamas', 'parado'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

In [52]:
# # No correr - script carpetas
# for action in actions: 
#     action_path = os.path.join(DATA_PATH, action)
#     if not os.path.exists(action_path):
#         os.makedirs(action_path)
    
#     dirmax = np.max(np.array(os.listdir(action_path)).astype(int)) if os.listdir(action_path) else 0

#     for sequence in range(0, no_sequences):
#         try: 
#             os.makedirs(os.path.join(action_path, str(dirmax + sequence)))
#         except Exception as e:
#             print(f"Error al crear el directorio: {e}")

In [53]:

# Function to collect data for a specific action
def collect_action_data(action_name, num_sequences=no_sequences, sequence_length=sequence_length):
    cap = cv2.VideoCapture(0)
    
    # Ensure action directory exists
    action_path = os.path.join(DATA_PATH, action_name)
    if not os.path.exists(action_path):
        os.makedirs(action_path)
    
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        # Loop through sequences aka videos
        for sequence in range(num_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_landmarks(image, results)

                # Apply wait logic on the first frame of each sequence
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_name, sequence), (15, 12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)  # Wait for 2 seconds before starting
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_name, sequence), (15, 12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints for each frame
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action_name, str(sequence), str(frame_num))
                if not os.path.exists(os.path.dirname(npy_path)):
                    os.makedirs(os.path.dirname(npy_path))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [60]:
#collect_action_data('parado')

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot o

In [62]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hola': 0,
 'chao': 1,
 'mi_nombre_es': 2,
 'como_estas': 3,
 'buenas_noches': 4,
 'INSOR': 5,
 'bien': 6,
 'mal': 7,
 'por_favor': 8,
 'como_te_llamas': 9,
 'parado': 10}